In [279]:
import pandas as pd
from pathlib import Path
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

from finta import TA
from finta.utils import resample_calendar
import json
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [280]:
# Alpaca for data
import alpaca_trade_api as api
from alpaca_trade_api.rest import TimeFrame

# pandas for analysis
import pandas as pd

# Plotly for charting
import plotly.graph_objects as go
import plotly.express as px

# Set default charting for pandas to plotly
pd.options.plotting.backend = "plotly"

In [281]:
import datetime as dt
import pytz
#today = dt.date.today()
# symbols we will be looking at
btc = "BTCUSD"
#spy = "SPY"

# start dates and end dates for backtest
#start_date = "2020-01-01"
#end_date = today 
#end_date = '2023-02-06'

end_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(hours=1)).isoformat()

start_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(days=1080)).isoformat()

# time frame for backtests
timeframe = TimeFrame.Day

In [282]:
# Our API keys for Alpaca
API_KEY = "PK5CD8L7ME7T2V9P1IB0"
API_SECRET = "N0bSGJwVRW4jLt0Uhj11DZEXyp7tnKP19FbcCz1P"

# Setup instance of alpaca api
alpaca = api.REST(API_KEY, API_SECRET)

# # # Request historical bar data for SPY and BTC using Alpaca Data API
# for equities, use .get_bars
#spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

# for crypto, use .get_crypto_bars, from multiple exchanges
btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df

btc_data = btc_data[btc_data['exchange'] == 'CBSE']

# display crypto bar data
display(btc_data)
#display(spy_data)

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-02-24 06:00:00+00:00,CBSE,9660.00,9669.99,9532.51,9554.77,1203.778464,12431,9590.793033
2020-02-25 06:00:00+00:00,CBSE,9304.99,9375.36,9092.92,9188.85,4503.851246,24892,9177.637936
2020-02-26 06:00:00+00:00,CBSE,9188.84,9280.00,8609.91,8778.30,15585.993780,79012,8894.716505
2020-02-27 06:00:00+00:00,CBSE,8814.90,8899.94,8779.06,8786.54,1819.421690,15034,8848.335058
2020-02-28 06:00:00+00:00,CBSE,8786.53,8813.12,8428.80,8708.89,11572.123138,63193,8624.928084
...,...,...,...,...,...,...,...,...
2023-02-03 06:00:00+00:00,CBSE,23527.18,23720.00,23208.01,23314.93,13387.860474,364786,23449.094528
2023-02-04 06:00:00+00:00,CBSE,23314.49,23589.67,23233.20,23382.81,3987.582539,232928,23381.304096
2023-02-05 06:00:00+00:00,CBSE,23382.81,23433.20,22637.51,22781.90,8385.600370,261845,22969.010291


In [283]:
ohlcv_df = btc_data.drop(columns=['exchange','trade_count','vwap'])
ohlcv_df

,open,high,low,close,volume
timestamp,,,,,
2020-02-24 06:00:00+00:00,9660.00,9669.99,9532.51,9554.77,1203.778464
2020-02-25 06:00:00+00:00,9304.99,9375.36,9092.92,9188.85,4503.851246
2020-02-26 06:00:00+00:00,9188.84,9280.00,8609.91,8778.30,15585.993780
2020-02-27 06:00:00+00:00,8814.90,8899.94,8779.06,8786.54,1819.421690
2020-02-28 06:00:00+00:00,8786.53,8813.12,8428.80,8708.89,11572.123138
...,...,...,...,...,...
2023-02-03 06:00:00+00:00,23527.18,23720.00,23208.01,23314.93,13387.860474
2023-02-04 06:00:00+00:00,23314.49,23589.67,23233.20,23382.81,3987.582539
2023-02-05 06:00:00+00:00,23382.81,23433.20,22637.51,22781.90,8385.600370


In [284]:
ohlc_df = ohlcv_df.drop(columns=['volume'])
ohlc_df

,open,high,low,close
timestamp,,,,
2020-02-24 06:00:00+00:00,9660.00,9669.99,9532.51,9554.77
2020-02-25 06:00:00+00:00,9304.99,9375.36,9092.92,9188.85
2020-02-26 06:00:00+00:00,9188.84,9280.00,8609.91,8778.30
2020-02-27 06:00:00+00:00,8814.90,8899.94,8779.06,8786.54
2020-02-28 06:00:00+00:00,8786.53,8813.12,8428.80,8708.89
...,...,...,...,...
2023-02-03 06:00:00+00:00,23527.18,23720.00,23208.01,23314.93
2023-02-04 06:00:00+00:00,23314.49,23589.67,23233.20,23382.81
2023-02-05 06:00:00+00:00,23382.81,23433.20,22637.51,22781.90


In [285]:
ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
ichimoku_df

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU
timestamp,,,,,
2020-02-24 06:00:00+00:00,NaN,NaN,NaN,NaN,6702.97
2020-02-25 06:00:00+00:00,NaN,NaN,NaN,NaN,6758.18
2020-02-26 06:00:00+00:00,NaN,NaN,NaN,NaN,6372.36
2020-02-27 06:00:00+00:00,NaN,NaN,NaN,NaN,6214.81
2020-02-28 06:00:00+00:00,NaN,NaN,NaN,NaN,6042.61
...,...,...,...,...,...
2023-02-03 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN
2023-02-04 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN
2023-02-05 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN


In [286]:
ichimoku_signal_df = pd.concat([ichimoku_df, ohlcv_df['close']], axis=1)
ichimoku_signal_df

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU,close
timestamp,,,,,,
2020-02-24 06:00:00+00:00,NaN,NaN,NaN,NaN,6702.97,9554.77
2020-02-25 06:00:00+00:00,NaN,NaN,NaN,NaN,6758.18,9188.85
2020-02-26 06:00:00+00:00,NaN,NaN,NaN,NaN,6372.36,8778.30
2020-02-27 06:00:00+00:00,NaN,NaN,NaN,NaN,6214.81,8786.54
2020-02-28 06:00:00+00:00,NaN,NaN,NaN,NaN,6042.61,8708.89
...,...,...,...,...,...,...
2023-02-03 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23314.93
2023-02-04 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23382.81
2023-02-05 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,22781.90


In [287]:

ichimoku_signal_df['signal'] = 0
ichimoku_signal_df["signal"] = np.where((ichimoku_signal_df["TENKAN"] > ichimoku_signal_df["KIJUN"]) &
                                (ichimoku_signal_df["close"] > ichimoku_signal_df["senkou_span_a"]), 1, 0)

ichimoku_signal_df['entry/exit'] = ichimoku_signal_df['signal'].diff()

ichimoku_signal_df


,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU,close,signal,entry/exit
timestamp,,,,,,,,
2020-02-24 06:00:00+00:00,NaN,NaN,NaN,NaN,6702.97,9554.77,0,NaN
2020-02-25 06:00:00+00:00,NaN,NaN,NaN,NaN,6758.18,9188.85,0,0.0
2020-02-26 06:00:00+00:00,NaN,NaN,NaN,NaN,6372.36,8778.30,0,0.0
2020-02-27 06:00:00+00:00,NaN,NaN,NaN,NaN,6214.81,8786.54,0,0.0
2020-02-28 06:00:00+00:00,NaN,NaN,NaN,NaN,6042.61,8708.89,0,0.0
...,...,...,...,...,...,...,...,...
2023-02-03 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23314.93,1,0.0
2023-02-04 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23382.81,1,0.0
2023-02-05 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,22781.90,1,0.0


In [288]:
ichimoku_signal_df['actual_returns'] = ichimoku_signal_df['close'].pct_change()
ichimoku_signal_df["strategy_returns"] = ichimoku_signal_df["actual_returns"] * ichimoku_signal_df["signal"].shift() 

In [289]:
ichimoku_signal_df

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU,close,signal,entry/exit,actual_returns,strategy_returns
timestamp,,,,,,,,,,
2020-02-24 06:00:00+00:00,NaN,NaN,NaN,NaN,6702.97,9554.77,0,NaN,NaN,NaN
2020-02-25 06:00:00+00:00,NaN,NaN,NaN,NaN,6758.18,9188.85,0,0.0,-0.038297,-0.000000
2020-02-26 06:00:00+00:00,NaN,NaN,NaN,NaN,6372.36,8778.30,0,0.0,-0.044679,-0.000000
2020-02-27 06:00:00+00:00,NaN,NaN,NaN,NaN,6214.81,8786.54,0,0.0,0.000939,0.000000
2020-02-28 06:00:00+00:00,NaN,NaN,NaN,NaN,6042.61,8708.89,0,0.0,-0.008837,-0.000000
...,...,...,...,...,...,...,...,...,...,...
2023-02-03 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23314.93,1,0.0,-0.009047,-0.009047
2023-02-04 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23382.81,1,0.0,0.002911,0.002911
2023-02-05 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,22781.90,1,0.0,-0.025699,-0.025699


In [290]:
actual_returns = ichimoku_signal_df['actual_returns'].cumsum()
actual_returns.max()

2.5458589911531213

In [291]:
ichimoku_signal_df

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU,close,signal,entry/exit,actual_returns,strategy_returns
timestamp,,,,,,,,,,
2020-02-24 06:00:00+00:00,NaN,NaN,NaN,NaN,6702.97,9554.77,0,NaN,NaN,NaN
2020-02-25 06:00:00+00:00,NaN,NaN,NaN,NaN,6758.18,9188.85,0,0.0,-0.038297,-0.000000
2020-02-26 06:00:00+00:00,NaN,NaN,NaN,NaN,6372.36,8778.30,0,0.0,-0.044679,-0.000000
2020-02-27 06:00:00+00:00,NaN,NaN,NaN,NaN,6214.81,8786.54,0,0.0,0.000939,0.000000
2020-02-28 06:00:00+00:00,NaN,NaN,NaN,NaN,6042.61,8708.89,0,0.0,-0.008837,-0.000000
...,...,...,...,...,...,...,...,...,...,...
2023-02-03 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23314.93,1,0.0,-0.009047,-0.009047
2023-02-04 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,23382.81,1,0.0,0.002911,0.002911
2023-02-05 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,22781.90,1,0.0,-0.025699,-0.025699


In [292]:
ichimoku_signal_df['entry/exit'].value_counts()

 0.0    1064
 1.0       8
-1.0       7
Name: entry/exit, dtype: int64

In [293]:
entry = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==1]['close'].hvplot.scatter(
    color='green',
    marker= '^'
)

exit = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==-1]['close'].hvplot.scatter(
    color='red',
    marker= 'v'
)

close = ichimoku_signal_df['close'].hvplot(
    color='lightgray',
    
)

actual_returns = actual_returns.hvplot(
    color='lightblue',
)

strategy_returns = ichimoku_signal_df['strategy_returns'].hvplot(
    color='lightgreen',
)

In [294]:
close * entry * exit * actual_returns * strategy_returns

:Overlay
   .Curve.Close            :Curve   [timestamp]   (close)
   .Scatter.Close.I        :Scatter   [timestamp]   (close)
   .Scatter.Close.II       :Scatter   [timestamp]   (close)
   .Curve.Actual_returns   :Curve   [timestamp]   (actual_returns)
   .Curve.Strategy_returns :Curve   [timestamp]   (strategy_returns)

In [295]:
"""
for index, row in ichimoku_signal_df.iterrows():
    if row["TENKAN"] > row["KIJUN"]:
        ichimoku_signal_df.loc[index, "Signal"] = 1.0
    if row["TENKAN"] < row["KIJUN"]:
        ichimoku_signal_df.loc[index,"Signal"] = -1.0

ichimoku_signal_df['entry/exit'] = ichimoku_signal_df['Signal'].diff()

ichimoku_signal_df
"""

'\nfor index, row in ichimoku_signal_df.iterrows():\n    if row["TENKAN"] > row["KIJUN"]:\n        ichimoku_signal_df.loc[index, "Signal"] = 1.0\n    if row["TENKAN"] < row["KIJUN"]:\n        ichimoku_signal_df.loc[index,"Signal"] = -1.0\n\nichimoku_signal_df[\'entry/exit\'] = ichimoku_signal_df[\'Signal\'].diff()\n\nichimoku_signal_df\n'

In [296]:
#ichimoku_signal_df.loc['entry/exit']  == 0

In [297]:
ichimoku_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [298]:
volume_splice_df['signal'] = np.nan

volume_splice_df.loc[(volume_splice_df['vwap']>= volume_splice_df['close']), 'vwap_signal'] = 1

volume_splice_df.loc[(volume_splice_df['vwap'] < volume_splice_df['close']), 'vwap_signal'] = -1

volume_splice_df["Entry/Exit"] = volume_splice_df["vwap_signal"].diff()

volume_splice_df['vwap_diff_from_close'] = volume_splice_df['close'] - volume_splice_df['vwap']

volume_splice_df.hvplot()

NameError: name 'volume_splice_df' is not defined

In [ ]:
volume_splice_df.hvplot()

In [ ]:
btc_data.hvplot()

In [ ]:
# Keep data from only CBSE exchange
btc_data = btc_data[btc_data['exchange'] == 'CBSE'] 

# keep only the daily close data column
btc_data = btc_data.filter(['close'])

# rename our close column to BTC
btc_data.rename(columns={'close':'BTC'}, inplace=True)

# keep only the date part of our timestamp index
btc_data.index = btc_data.index.map(lambda timestamp : timestamp.date)

# Clean SPY data
#spy_data = spy_data.filter(['close'])
#spy_data.rename(columns={'close':'SPY'}, inplace=True)
#spy_data.index = spy_data.index.map(lambda timestamp : timestamp.date)

In [ ]:
btc_data.hvplot()

In [ ]:
# Starting out my signals dataframe
signals_df = four_hr_ohlcv.loc[:,['close']]
# Generating returns from the BTC close prices using pct_change
signals_df['Actual Returns'] = signals_df['close'].pct_change().dropna()
# Adding more indicators to the sisgnals_df
signals_df['RSI63'] = TA.RSI(four_hr_ohlcv,63)
signals_df.dropna()
# Checking the RSI plot
RSI63 = signals_df['RSI63'].hvplot()



In [ ]:
# making the sma  rolling window for volume
volume_sma = 7
signals_df['volume'] = four_hr_ohlcv['volume']

# making the sma based on the rolling window for volume
signals_df['Volume SMA 7'] = signals_df['volume'].rolling(window=volume_sma).mean()
signals_df.dropna()

# Checking the Volume SMA plot
Volume_SMA_7 = signals_df['Volume SMA 7'].hvplot()
Volume = signals_df['volume'].hvplot.bar()
Volume

